In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json, os, collections, ast

In [2]:
playtest_files = ["2018-11-14_Playtest/anonymized_playtest5_data.csv", "2019-01-07_Playtest/anonymized_playtest6_data.csv", "2019-01-31_Playtest/anonymized_playtest7_data.csv"]

playtests = []

for playtest in playtest_files:
    playtests.append(pd.read_csv(playtest, sep=";", index_col=0))
    
# Set the playtest index we want to focus on
playtest = 1


In [3]:
def get_shapes(puzzle):
    shapes = []
    if puzzle is None:
        return shapes
    for shape_data in puzzle.get("shapeData"):
        shapes.append(shape_data.get('shapeType'))
    return shapes    

In [4]:
puzzles = collections.OrderedDict()

with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/config.json") as f:
    asset_config = json.load(f)

for puzzle_sets in asset_config.get("puzzleSets"):
    for puzzle_file in puzzle_sets.get("puzzles"):
        with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/{puzzle_file}.json") as f:
            puzzle_details = json.load(f)
            puzzles[puzzle_details.get("puzzleName")] = puzzle_details

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
#all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])
all_playtest_data = pd.concat([playtests[playtest]])

#all_playtest_data = all_playtest_data.join(all_playtest_data['data'].map(json.loads).apply(pd.Series).add_prefix('data.')).drop(["data"], axis=1)
all_playtest_data = all_playtest_data.join(all_playtest_data['data'].map(json.loads).apply(pd.Series)).drop(["data"], axis=1)

In [6]:
# puzzleDict[user~puzzle_id] = {}
puzzleDict = {}
frames = []

# For each session
for user in all_playtest_data['session_id'].unique():
    # Get the sessions
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
    activePuzzle = None
    for enum, event in user_events.iterrows():
        #print(('{} - {}').format(event['time'], event['type']))
        # Keep track of all frames we've looked at
        if(event['type'] == 'ws-start_level'):
            activePuzzle = event['task_id']
            frames.clear()

            
        # If they are not playing a puzzle we do not do anything and continue
        if(activePuzzle is None):
            continue

    # Analyze when puzzle is finished or user left
        # Measure time, attempts, completion and actions
        if(event['type'] in ['ws-exit_to_menu', 'ws-disconnect']):
            # This was incomplete so don't reset the active puzzle
            activePuzzle = None
            frames.clear()

        elif (event['type'] in ['ws-puzzle_complete']):
            # print (event['task_id'])
            # This was a correct solution! We need to mark all of these frames as correct
            for frame in frames:
                all_playtest_data.at[frame, 'completed'] = True
            
        else:
            # Set the active puzzle to be this one we're looking at
            all_playtest_data.at[enum, 'task_id'] = activePuzzle
            frames.append(enum)


In [7]:
all_playtest_data['time'] = pd.to_datetime(all_playtest_data['time'])


In [8]:
all_attempts = all_playtest_data.task_id.value_counts()


In [9]:
complete_puzzles = all_playtest_data.loc[all_playtest_data['type'] == 'ws-puzzle_complete']
successful_attempts = complete_puzzles.task_id.value_counts()

In [10]:
created_shapes = all_playtest_data.query('type == "ws-create_shape"')
success_created_shapes =  created_shapes.query('completed == True')

shapes_used = collections.defaultdict(list)
for shapes, value in created_shapes.iterrows():
    shapes_used[value.get('task_id')].append(int(value.get('shapeType')))

print (success_created_shapes)
shapes_used_success = collections.defaultdict(list)
for shapes, value in success_created_shapes.iterrows():
    shapes_used_success[value.get('task_id')].append(int(value.get('shapeType')))

                            time             type  \
id                                                  
24168 2019-01-07 10:50:01.583632  ws-create_shape   
24217 2019-01-07 13:39:53.379902  ws-create_shape   
24233 2019-01-07 13:40:06.595611  ws-create_shape   
24237 2019-01-07 13:40:11.316359  ws-create_shape   
24275 2019-01-07 13:42:56.048394  ws-create_shape   
24276 2019-01-07 13:42:57.885588  ws-create_shape   
24296 2019-01-07 13:43:12.355315  ws-create_shape   
24299 2019-01-07 13:43:13.750324  ws-create_shape   
24314 2019-01-07 13:43:18.049101  ws-create_shape   
24332 2019-01-07 13:43:29.209542  ws-create_shape   
24351 2019-01-07 13:43:33.548973  ws-create_shape   
24353 2019-01-07 13:43:34.543875  ws-create_shape   
24354 2019-01-07 13:43:35.201886  ws-create_shape   
24362 2019-01-07 13:43:37.943137  ws-create_shape   
24369 2019-01-07 13:43:41.543759  ws-create_shape   
24371 2019-01-07 13:43:42.017110  ws-create_shape   
24375 2019-01-07 13:43:43.486495  ws-create_sh

In [11]:
for attempt, value in all_attempts.items():
    print(attempt)
    success_value = successful_attempts.get(attempt, 0)
    shapes = get_shapes(puzzles.get(attempt))
    print (f"Puzzle {attempt: <25} Attempts {value: < 5} / Success {success_value: < 5} : {round(((success_value/value) * 100), 1): < 5}% Solution Shapes: {shapes}")
    times_used = collections.Counter(shapes_used[attempt])
    print (f"Shapes Used Total: {times_used}")
    times_used = collections.Counter(shapes_used_success[attempt])
    print (f"Shapes Used Success: {times_used}")


Design a Puzzle
Puzzle Design a Puzzle           Attempts  1257 / Success  0    :  0.0 % Solution Shapes: []
Shapes Used Total: Counter({1: 33, 4: 23, 5: 18, 2: 11, 3: 6, 6: 5})
Shapes Used Success: Counter({1: 3, 4: 3, 2: 3, 5: 1, 3: 1})
Pyramids are Strange
Puzzle Pyramids are Strange      Attempts  1050 / Success  6    :  0.6 % Solution Shapes: [2, 2, 2, 2]
Shapes Used Total: Counter({2: 48})
Shapes Used Success: Counter({2: 31})
Scaling Round Objects
Puzzle Scaling Round Objects     Attempts  767  / Success  7    :  0.9 % Solution Shapes: [4, 2]
Shapes Used Total: Counter({4: 13, 5: 13, 6: 1, 1: 1})
Shapes Used Success: Counter({4: 10, 5: 9, 6: 1})
Cubes Obscure Spheres
Puzzle Cubes Obscure Spheres     Attempts  724  / Success  6    :  0.8 % Solution Shapes: [6, 4, 4, 1]
Shapes Used Total: Counter({1: 33, 4: 27, 6: 11})
Shapes Used Success: Counter({1: 14, 4: 13, 6: 7})
Not Bird
Puzzle Not Bird                  Attempts  647  / Success  3    :  0.5 % Solution Shapes: [6, 4, 2]
Shap

In [12]:

all_playtest_data.type.value_counts()

ws-mode_change                1520
ws-rotate_shape               1202
ws-rotate_view                1088
ws-scale_shape                1004
ws-move_shape                 958 
ws-select_shape               927 
ws-deselect_shape             832 
ws-create_shape               649 
ws-puzzle_started             267 
ws-start_level                245 
ws-check_solution             243 
ws-click_nothing              237 
ws-exit_to_menu               200 
ws-snapshot                   194 
ws-delete_shape               158 
ws-puzzle_complete            143 
ws-undo_action                56  
ws-toggle_snapshot_display    50  
ws-restart_puzzle             22  
ws-start_game                 22  
ws-login_user                 22  
ws-select_shape_add           20  
ws-paint                      14  
ws-toggle_paint_display       12  
ws-palette_change             12  
ws-redo_action                3   
start_game                    1   
Name: type, dtype: int64